## Corruption demo

In [1]:
import pandas as pd
import numpy as np
import random
import re
import string

from splink.datasets import splink_datasets
from splink.duckdb.linker import DuckDBLinker
from splink.duckdb.blocking_rule_library import block_on
import splink.duckdb.comparison_template_library as ctl
import splink.duckdb.comparison_library as cl

import corrupter

In [2]:
def add_match_labels(df, match_status):
    df_new = df.copy()
    if match_status == "matches":
        df_new['id_l_prefix'] = df_new['unique_id_l'].str.split('-').str[0]
        df_new['id_r_prefix'] = df_new['unique_id_r'].str.split('-').str[0]
        df_new['true_match'] = (df_new['id_l_prefix'] == df_new['id_r_prefix']).astype(int)
        df_new = df_new.drop(columns=['id_l_prefix', 'id_r_prefix'])
    elif match_status == "non-matches":
        df_new['true_match'] = 0
    df_new['predict_high_match'] = df_new['match_probability'].apply(lambda x: 0 if x < 0.999 else 1)
    df_new['predict_med_match'] = df_new['match_probability'].apply(lambda x: 0 if x < 0.99 else 1)
    df_new['predict_low_match'] = df_new['match_probability'].apply(lambda x: 0 if x < 0.95 else 1)
    return df_new

In [3]:
def predict_scores(df, model, match_status):
    linker = DuckDBLinker(df, model)
    df_predict = linker.predict().as_pandas_dataframe()
    df_predict = add_match_labels(df_predict, match_status)
    return df_predict

### Data creation

Create the base dataset. This is a set of records which will form the base of the prediction, as well as providing the base for corruption.

This dataset needs to match the dataset used to train the model (completed in a separate file). The specific way in which the base records are selected is specific to the dataset. This section could provide an example of some 'perfect' records. 

In [4]:
df = splink_datasets.historical_50k
#df = splink_datasets.fake_1000
#df = splink_datasets.febrl3

In [5]:
# H50K
# Select one example per individual
person_df = df.groupby('cluster').first().reset_index()
# Drop any rows with NA values (we do not want these in our perfect data)
person_df = person_df.dropna()
# Filter out rows where name contains a number or a full stop
pattern = r'[.\d\-,]'
person_df = person_df[~person_df['full_name'].str.contains(pattern)]
# Take a random sample of 10 people 
base_df = person_df.sample(10, random_state=7)

In [6]:
# # F1K
# # Select one example per individual
# person_df = df.groupby('cluster').first().reset_index()
# # Drop any rows with NA values (we do not want these in our perfect data)
# person_df = person_df.dropna()
# # Take a random sample of 10 people 
# base_df = person_df.sample(10, random_state=7)

In [7]:
# # FEBRL3
# df.replace(' ', np.nan, inplace=True)
# person_df = df.dropna()
# # Take a random sample of 10 people 
# base_df = person_df.sample(10, random_state=10)

Create a 'perfect' dataset which will be a full duplication of the base records (to simulate 10 perfectly matching records).

In [8]:
# H50K
base_df_dupe = base_df.copy()
base_df_dupe['unique_id'] = base_df_dupe['unique_id'].str.replace('-1', '-2')
perfect_df = pd.concat([base_df, base_df_dupe], ignore_index=True)

In [9]:
# # F1K
# base_df_dupe = base_df.copy()
# base_df['unique_id'] = base_df['unique_id'].astype(str) + '_1'
# base_df_dupe['unique_id'] = base_df_dupe['unique_id'].astype(str) + '_2'
# perfect_df = pd.concat([base_df, base_df_dupe], ignore_index=True)

### Corruption

Create the lists of optional replacements which the necessary corrupter functions require. These can differ based on what source the user wants them to come from. In this example, there's no reason for these lists to differ from values in the base df. However, if you were changing names to specific name types (i.e. non-anglicised), you may want to load a specific list of names. 

In [10]:
list_of_surnames = person_df['surname']
list_of_postcodes = person_df['postcode_fake']
list_of_first_names = person_df['first_name']
list_of_occupations = person_df['occupation']
list_of_dobs = person_df['dob']
#list_of_cities = person_df['city']

Corruption fields can be defined. In this example, I am defining several as results will be assessed separately. However, it is possible that users may only be interested in one or two. 
There are options to apply multiple corruptions to one column or multiple corruptions to one row. However, with this current set up, a single corruption cannot include two columns (for example, name inversion). 

In [46]:
surname_corruption = {'surname': [corrupter.corrupt_random_swap(list_of_surnames)]}
postcode_partial_corruption = {'postcode_fake': [corrupter.corrupt_postcode(list_of_postcodes)]}

first_name_db_sp_corruption = {'first_name': [corrupter.corrupt_name(list_of_first_names), corrupter.corrupt_spelling(string_type="char")]}

surname_postcode_corruption = {
    'surname': [corrupter.corrupt_random_swap(list_of_surnames)],
    'postcode_fake': [corrupter.corrupt_postcode(list_of_postcodes)]
}

firstname_occupation_corruption = {
    'first_name': [corrupter.corrupt_random_swap(list_of_first_names)], 
    'occupation': [corrupter.corrupt_random_swap(list_of_occupations)]
    }

In [47]:
corrupted_surname_df = corrupter.apply_corrupt_to_df(base_df, surname_corruption)
corrupted_postcode_partial_df = corrupter.apply_corrupt_to_df(base_df, postcode_partial_corruption)

corrupted_first_name_db_sp_df = corrupter.apply_corrupt_to_df(base_df, first_name_db_sp_corruption)

corrupted_surname_postcode_df = corrupter.apply_corrupt_to_df(base_df, surname_postcode_corruption)

corrupted_firstname_occupation_df = corrupter.apply_corrupt_to_df(base_df, firstname_occupation_corruption)

### Prediction

Create a base linker in case any part of the model configuration is to be inspected (don't love this).

In [13]:
linker = DuckDBLinker(df, "model_h50k.json")
linker.match_weights_chart()

In [48]:
perfect_predict_df = predict_scores(perfect_df, "model_h50k.json", match_status='matches')

corrupted_surname_predict_df = predict_scores(corrupted_surname_df, "model_h50k.json", match_status='matches')
corrupted_postcode_partial_predict_df = predict_scores(corrupted_postcode_partial_df, "model_h50k.json", match_status='matches')

corrupted_first_name_db_sp_predict_df = predict_scores(corrupted_first_name_db_sp_df, "model_h50k.json", match_status='matches')

corrupted_surname_postcode_predict_df = predict_scores(corrupted_surname_postcode_df, "model_h50k.json", match_status="matches")

corrupted_firstname_occupation_predict_df = predict_scores(corrupted_firstname_occupation_df, "model_h50k.json", match_status='non-matches')

### Analysis

In [26]:
linker_perfect = DuckDBLinker(perfect_df, "model_h50k.json")
perfect_predict = linker_perfect.predict()
perfect_records_to_view = perfect_predict.as_record_dict()

linker_perfect.waterfall_chart(perfect_records_to_view)

alt.LayerChart(...)

In [49]:
linker_corrupt = DuckDBLinker(corrupted_surname_postcode_df, "model_h50k.json")
corrupt_predict = linker_corrupt.predict()
corrupt_records_to_view = corrupt_predict.as_record_dict()

linker_corrupt.waterfall_chart(corrupt_records_to_view)

alt.LayerChart(...)

In [50]:
corrupted_surname_postcode_predict_df

,match_weight,match_probability,unique_id_l,unique_id_r,first_name_l,first_name_r,gamma_first_name,tf_first_name_l,tf_first_name_r,bf_first_name,...,gamma_occupation,tf_occupation_l,tf_occupation_r,bf_occupation,bf_tf_adj_occupation,match_key,true_match,predict_high_match,predict_med_match,predict_low_match
0,5.756657,0.981840,Q35610-1,Q35610-2,arthur,arthur,4,0.2,0.2,44.511481,...,1,0.1,0.1,22.121353,0.405239,2,1,0,0,1
1,5.756657,0.981840,Q26732214-1,Q26732214-2,frederic,frederic,4,0.1,0.1,44.511481,...,1,0.1,0.1,22.121353,0.405239,2,1,0,0,1
2,5.756657,0.981840,Q43136267-1,Q43136267-2,frank,frank,4,0.1,0.1,44.511481,...,1,0.1,0.1,22.121353,0.405239,2,1,0,0,1
3,6.756657,0.990837,Q2551919-1,Q2551919-2,graham,graham,4,0.1,0.1,44.511481,...,1,0.1,0.1,22.121353,0.405239,2,1,0,1,1
4,6.756657,0.990837,Q6253670-1,Q6253670-2,john,john,4,0.1,0.1,44.511481,...,1,0.1,0.1,22.121353,0.405239,2,1,0,1,1
5,6.756657,0.990837,Q15971359-1,Q15971359-2,walter,walter,4,0.1,0.1,44.511481,...,1,0.1,0.1,22.121353,0.405239,2,1,0,1,1
6,6.756657,0.990837,Q5401176-1,Q5401176-2,esther,esther,4,0.1,0.1,44.511481,...,1,0.1,0.1,22.121353,0.405239,2,1,0,1,1
7,6.756657,0.990837,Q86260965-1,Q86260965-2,william,william,4,0.1,0.1,44.511481,...,1,0.1,0.1,22.121353,0.405239,2,1,0,1,1
8,6.756657,0.990837,Q21557354-1,Q21557354-2,lucy,lucy,4,0.1,0.1,44.511481,...,1,0.1,0.1,22.121353,0.405239,2,1,0,1,1
9,7.218601,0.993331,Q21458727-1,Q21458727-2,arthur,arthur,4,0.2,0.2,44.511481,...,1,0.1,0.1,22.121353,0.405239,2,1,0,1,1
